In [1]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix

In [2]:
# 导入
import numpy as np
import pandas as pd

# 读取数据
df = pd.read_csv("error1.csv", index_col=0) #M101 test
df

,日期,时间,生产线编号,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,...,不合格数,物料推送装置故障1001,物料检测装置故障2001,填装装置检测故障4001,填装装置定位故障4002,填装装置填装故障4003,加盖装置定位故障5001,加盖装置加盖故障5002,拧盖装置定位故障6001,拧盖装置拧盖故障6002
41066,2,12236,M101,0,1,1836,1836,614,613,611,...,0,0,0,0,0,0,0,0,0,6002
41067,2,12237,M101,0,1,1836,1836,614,613,611,...,0,0,0,0,0,0,0,0,0,6002
41068,2,12238,M101,0,1,1836,1836,614,613,611,...,0,0,0,0,0,0,0,0,0,6002
41069,2,12239,M101,0,1,1836,1836,614,613,611,...,0,0,0,0,0,0,0,0,0,6002
41070,2,12240,M101,0,1,1836,1836,614,613,611,...,0,0,0,0,0,0,0,0,0,6002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7443001,360,732,M101,0,1,78,78,37,36,25,...,7,0,0,4001,0,0,0,0,0,0
7443002,360,733,M101,0,1,78,78,37,36,25,...,7,0,0,4001,0,0,0,0,0,0
7443003,360,734,M101,0,1,78,78,37,36,25,...,7,0,0,4001,0,0,0,0,0,0
7443004,360,735,M101,0,1,78,78,37,36,25,...,7,0,0,4001,0,0,0,0,0,0


In [3]:
wrong_list=['物料推送装置故障1001',
'物料检测装置故障2001',
'填装装置检测故障4001',
'填装装置定位故障4002',
'填装装置填装故障4003',
'加盖装置定位故障5001',
'加盖装置加盖故障5002',
'拧盖装置定位故障6001',
'拧盖装置拧盖故障6002'
]
def check_non_zero(row):
    # 仅检查指定列中是否有非零值
    non_zero = row[wrong_list].replace(0, pd.NA).notna().any()
    return 1 if non_zero else 0

In [4]:
for i in wrong_list:
    print(df[i].value_counts())


物料推送装置故障1001
0       33840
1001     7639
Name: count, dtype: int64
物料检测装置故障2001
0       38797
2001     2682
Name: count, dtype: int64
填装装置检测故障4001
0       38135
4001     3344
Name: count, dtype: int64
填装装置定位故障4002
0       36168
4002     5311
Name: count, dtype: int64
填装装置填装故障4003
0       36118
4003     5361
Name: count, dtype: int64
加盖装置定位故障5001
0       36130
5001     5349
Name: count, dtype: int64
加盖装置加盖故障5002
0       37934
5002     3545
Name: count, dtype: int64
拧盖装置定位故障6001
0       38536
6001     2943
Name: count, dtype: int64
拧盖装置拧盖故障6002
0       36174
6002     5305
Name: count, dtype: int64


In [5]:
df['error']=df[wrong_list].apply(check_non_zero, axis=1)
df = df.drop(wrong_list, axis=1)
error_counts = df['error'].value_counts()
error_counts

error
1    41479
Name: count, dtype: int64

In [6]:
df = df.drop(['生产线编号','日期'], axis=1)
df.set_index('时间', inplace=True)
df

,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,填装定位器固定状态,填装定位器放开状态,物料抓取数,...,加盖下降数,加盖数,拧盖检测数,拧盖定位数,拧盖下降数,拧盖旋转数,拧盖数,合格数,不合格数,error
时间,,,,,,,,,,,,,,,,,,,,,
12236,0,1,1836,1836,614,613,611,0,1,1833,...,610,610,610,610,609,609,609,609,0,1
12237,0,1,1836,1836,614,613,611,0,1,1833,...,610,610,610,610,609,609,609,609,0,1
12238,0,1,1836,1836,614,613,611,0,1,1833,...,610,610,610,610,609,609,609,609,0,1
12239,0,1,1836,1836,614,613,611,0,1,1833,...,610,610,610,610,609,609,609,609,0,1
12240,0,1,1836,1836,614,613,611,0,1,1833,...,610,610,610,610,609,609,609,609,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,0,1,78,78,37,36,25,0,1,75,...,33,33,33,33,33,33,33,25,7,1
733,0,1,78,78,37,36,25,0,1,75,...,33,33,33,33,33,33,33,25,7,1
734,0,1,78,78,37,36,25,0,1,75,...,33,33,33,33,33,33,33,25,7,1


In [13]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
n = len(df.columns)
# 假设你的数据集是一个Pandas DataFrame，名为df，且已经进行了适当的预处理
df1 = df.iloc[:, :n-1]
# 数据标准化
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df1)

# 选择聚类的数量
k = 9  # 假设我们想要的聚类数为10
# 初始化KMeans模型
# n_clusters是要形成的簇的数量，即类别数
# init是选择初始簇中心的方法，'k-means++'是一种可以加速收敛的方法
# n_init是算法运行具有不同质心种子的次数
# max_iter是算法的最大迭代次数
# tol是收敛的容忍度
kmeans = KMeans(
    n_clusters=k, 
    init='k-means++', 
    n_init=10, 
    max_iter=1000, 
    tol=0.001, 
    random_state=42
)

# 拟合模型
kmeans.fit(scaled_features)

# 预测聚类
clusters = kmeans.predict(scaled_features)

# 将聚类结果添加到原始数据集中
df1['cluster'] = clusters

# 查看聚类结果
df1['cluster'].value_counts()


cluster
2    7264
4    6597
5    6225
3    5357
6    4649
1    4134
0    3782
8    3450
7      21
Name: count, dtype: int64

In [9]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import pandas as pd

# 假设df是您的数据框架，其中包含您的特征
# 数据预处理：这里可以根据需要添加

# 初始化层次聚类模型
# n_clusters是要形成的簇的数量
# metric是计算距离的指标
# linkage是连接簇的准则
# ‘ward’: 最小化簇内的方差，适合于球形簇。
# ‘complete’: 最大距离连接，考虑簇中最远的点。
# ‘average’: 平均距离连接，考虑簇中所有点的平均距离。
# ‘single’: 最小距离连接，考虑簇中最近的点。
hierarchical = AgglomerativeClustering(
    n_clusters=9, 
    metric='euclidean', 
    linkage='complete'
)

# 拟合模型并预测类别
clusters = hierarchical.fit_predict(df1)

# 将类别结果添加到原始数据框架中
df1['cluster'] = clusters

df1['cluster'].value_counts()


cluster
0    6360
2    5569
1    5529
3    5340
4    4580
6    4141
5    4103
8    2953
7    2904
Name: count, dtype: int64

In [10]:
# 4274

# 504

# 1511

# 2933

# 2712

# 2545

# 2209

# 2029

# 2661

2661